In [1]:
import ee
import geemap

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

Map = geemap.Map(center=[0, 0], zoom=2)
Map.add_basemap("HYBRID")
Map.addLayerControl()  # To toggle layers on/off
Map



Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

<IPython.core.display.Javascript object>

style = {'color': '0000007f', 'width': 2, 'lineType': 'solid', 'fillColor': '000000a8'}


In [9]:
if Map.user_rois is None:
    print("No user ROI found. Please draw an ROI on the map before running this cell.")
else:
    # Check the number of features in the collection
    roi_size = Map.user_rois.size().getInfo()
    if roi_size == 0:
        print("No user ROI found. Please draw an ROI on the map before running this cell.")
    else:
        # We'll take the first feature from the FeatureCollection and get its geometry
        first_feature = Map.user_rois.first()
        roi = first_feature.geometry()
        print("ROI successfully retrieved!")
        print("Feature info:", first_feature.getInfo())
        


ROI successfully retrieved!
Feature info: {'type': 'Feature', 'geometry': {'geodesic': False, 'type': 'Polygon', 'coordinates': [[[-119.553223, 32.944149], [-119.553223, 34.83635], [-116.103516, 34.83635], [-116.103516, 32.944149], [-119.553223, 32.944149]]]}, 'id': '0', 'properties': {}}


In [10]:
import datetime

start_date = '2024-09-01'

end_date = datetime.datetime.now().strftime('%Y-%m-%d')

In [13]:
s2_collection = (
    ee.ImageCollection("COPERNICUS/S2_SR")
    .filterBounds(roi)
    .filterDate(start_date, end_date)
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 20))
)

In [16]:
s2_median = s2_collection.median()

In [17]:
ndvi = s2_median.normalizedDifference(["B8", "B4"]).rename("NDVI")

In [18]:
forest_mask = ndvi.gt(0.6)
shrub_mask = ndvi.gt(0.3).And(ndvi.lte(0.6))


In [19]:
classification = (forest_mask.multiply(1).add(shrub_mask.multiply(2))).rename("landcover")


In [20]:
fire_potential = ee.Image(1).subtract(ndvi).rename("fire_potential")


In [21]:
analysis_image = s2_median.select(["B4", "B3", "B2"]) \
    .addBands([classification, fire_potential, ndvi]) \
    .clip(roi)

In [22]:
rgb_vis = {
    "bands": ["B4", "B3", "B2"],
    "min": 0,
    "max": 3000,
}


In [23]:
class_vis = {
    "min": 0, 
    "max": 2, 
    "palette": ["ffffff", "006400", "7FFF00"]  # white=none, darkgreen=forest, greenyellow=shrub
}


In [24]:
fire_vis = {
    "min": 0,
    "max": 1,
    "palette": ["blue", "yellow", "red"] 
}


In [25]:
Map.addLayer(analysis_image, rgb_vis, "S2 Composite")
Map.addLayer(analysis_image.select("landcover"), class_vis, "Forest/Shrubland")
Map.addLayer(analysis_image.select("fire_potential"), fire_vis, "Fire Potential")
Map.centerObject(roi, 7)
Map


Map(bottom=6782.0, center=[33.89075720196369, -117.82836949999998], controls=(WidgetControl(options=['position…

In [49]:
task = ee.batch.Export.image.toDrive(
    image=analysis_image.float(),
    description="S2_Fire_Analysis",
    folder="GEE_Exports",
    fileNamePrefix="s2_fire_analysis",
    region=roi,
    scale=50, 
    crs="EPSG:4326"
)

task.start()
print("Export started. Check Tasks tab in the Code Editor or your GEE console.")


Export started. Check Tasks tab in the Code Editor or your GEE console.


In [60]:
status = task.status()
print(status)


{'state': 'COMPLETED', 'description': 'S2_Fire_Analysis', 'priority': 100, 'creation_timestamp_ms': 1736421845128, 'update_timestamp_ms': 1736422517368, 'start_timestamp_ms': 1736421853147, 'task_type': 'EXPORT_IMAGE', 'destination_uris': ['https://drive.google.com/#folders/1bDxH5MT-WvX_gkAs8J3eecp1vC9vxavN'], 'attempt': 1, 'batch_eecu_usage_seconds': 2483.1298828125, 'id': 'CIXGLCJTKZA2DDPQKTW42627', 'name': 'projects/earthengine-legacy/operations/CIXGLCJTKZA2DDPQKTW42627'}


In [61]:
import rasterio
import numpy as np

In [33]:
s3_tif = r".\2025-01-08-00_00_2025-01-08-23_59_Sentinel-3_SLSTR_F1_Brightness_Temperature.tiff"

In [62]:
gee_tif = r".\s2_fire_analysis-0000000000-0000000000.tif"

In [71]:
s3_profile["crs"], s3_profile["transform"], s3_profile["width"], s3_profile["height"]

(CRS.from_wkt('GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]'),
 Affine(0.0026940764667106177, 0.0, -119.388428,
        0.0, -0.0022413152985074625, 34.930979),
 1517,
 1072)

In [80]:
from rasterio.warp import calculate_default_transform, reproject, Resampling

# Get the Sentinel-3 raster profile
with rasterio.open(s3_tif) as s3_src:
    s3_profile = s3_src.profile.copy()
    s3_crs = s3_profile["crs"]
    s3_transform = s3_profile["transform"]
    s3_width = s3_profile["width"]
    s3_height = s3_profile["height"]

# Reproject each band of the S2 analysis to match S3
with rasterio.open(gee_tif) as s2_src:
    s2_profile = s2_src.profile.copy()
    # Number of bands
    s2_band_count = s2_src.count

    # Prepare an array to hold all reprojected bands
    reprojected_s2_data = np.zeros((s2_band_count, s3_height, s3_width), dtype=s2_src.dtypes[0])

    for band_i in range(1, s2_band_count+1):
        band_data = s2_src.read(band_i)

        reproject(
            source=band_data,
            destination=reprojected_s2_data[band_i-1],
            src_transform=s2_src.transform,
            src_crs=s2_src.crs,
            dst_transform=s3_transform,
            dst_crs=s3_crs,
            resampling=Resampling.nearest
        )


In [88]:
b4_idx = 0
b3_idx = 1
b2_idx = 2
landcover_idx = 3
firepot_idx   = 4
ndvi_idx      = 5

# Now these should have shape (s3_height, s3_width)
landcover_data = reprojected_s2_data[landcover_idx, :, :]
firepot_data   = reprojected_s2_data[firepot_idx, :, :]


In [89]:
with rasterio.open(s3_tif) as src:
    s3_profile = src.profile.copy()
    s3_data = src.read(1)  # assuming single band
    no_data_val = s3_profile.get('nodata', None)

# We want all pixels with Temperature > 50 C.
temp_threshold = 50  
temp_mask = (s3_data > temp_threshold)

In [90]:
with rasterio.open(gee_tif) as src:
    gee_profile = src.profile.copy()
    # This image has multiple bands: B4, B3, B2, landcover, fire_potential, NDVI
    # So we read them all:
    gee_data = src.read()  # shape: (bands, rows, cols)


1: B4
2: B3
3: B2
4: landcover
5: fire_potential
6: NDVI


In [91]:
high_fire_potential_mask = (firepot_data > 0.7) & (landcover_data >= 1)


In [92]:
hot_and_high_fire_risk = temp_mask & high_fire_potential_mask


In [93]:
import matplotlib
import geemap
import numpy as np

b4 = gee_data[0].astype(np.float32)
b3 = gee_data[1].astype(np.float32)
b2 = gee_data[2].astype(np.float32)

def stretch_band(band, min_val=0, max_val=3000):
    band = np.clip(band, min_val, max_val)
    return (band - min_val) / (max_val - min_val)

r = stretch_band(b4)
g = stretch_band(b3)
b = stretch_band(b2)

# Combine into an RGB array for visualization: shape = (rows, cols, 3)
rgb_vis_array = np.dstack([r, g, b])


In [94]:
rows, cols = hot_and_high_fire_risk.shape
overlay_rgba = np.zeros((rows, cols, 4), dtype=np.float32)

# Where condition is True, paint red
overlay_rgba[hot_and_high_fire_risk, 0] = 1.0  # Red channel
overlay_rgba[hot_and_high_fire_risk, 3] = 0.4  # alpha=0.4 for partial transparency


In [99]:
import rasterio
from rasterio.transform import from_origin

# Suppose 'rgb_vis_array' has shape (rows, cols, 3) for an RGB image,
# and we know the bounding box [xmin, ymin, xmax, ymax].
# We'll define a transform and CRS for a simple example:
xmin, ymax = -119.553223, 34.83635
xmax, ymin = -116.103516, 32.944149
cols = rgb_vis_array.shape[1]
rows = rgb_vis_array.shape[0]
# Create an affine transform for bounding box in EPSG:4326
xres = (xmax - xmin) / cols
yres = (ymax - ymin) / rows
transform = from_origin(xmin, ymax, xres, yres)

out_tif = "rgb_visual.tif"
with rasterio.open(
    out_tif,
    "w",
    driver="GTiff",
    height=rows,
    width=cols,
    count=3,            # three bands for RGB
    dtype=rgb_vis_array.dtype,
    crs="EPSG:4326",
    transform=transform,
) as dst:
    # rasterio expects the shape as (bands, rows, cols)
    for b_i in range(3):
        dst.write(rgb_vis_array[:,:,b_i], b_i+1)


In [106]:
Map = geemap.Map(center=[33.9, -117.8], zoom=7)


In [109]:
tif_path = r".\rgb_visual.tif"
Map.add_raster(tif_path, layer_name="RGB Visual")


In [111]:
Map

Map(center=[33.8902495, -117.82836950000001], controls=(WidgetControl(options=['position', 'transparent_bg'], …